In [1]:
# import sys
# !{sys.executable} -m pip install chardet
# import gdal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import chardet
# import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
print(tf.__version__)

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

# raw_dataset = pd.read_csv("/data/fast0/datasets/glathida-3.1.0/data/T.csv")
# raw_dataset = pd.read_csv("~/stuff/coding/glacier/data/T.csv")


#examine data columns
# dataset.head()

2.7.0


# Step 1: Organize the data

## A. Load and organize the Glathida dataset

In [2]:
# raw_dataset = pd.read_csv('/data/fast0/datasets/glathida-3.1.0/data/T.csv')
raw_dataset = pd.read_csv('~/data/glac/T_models/T.csv')
glathida = raw_dataset.copy()
glathida = glathida.drop([
#                         'GlaThiDa_ID',
#                         'GLACIER_DB',
#                         'GLACIER_ID',
                        'POLITICAL_UNIT',
                        'GLACIER_NAME',
                        'SURVEY_DATE',
                        'MAXIMUM_THICKNESS',
                        'MAX_THICKNESS_UNCERTAINTY',
                        'DATA_FLAG',
                        'ELEVATION_DATE',
                        'SPONSORING_AGENCY',
                        'REMARKS',
                        'SURVEY_METHOD_DETAILS',
                        'SURVEY_METHOD',
                        'NUMBER_OF_SURVEY_POINTS',
                        'NUMBER_OF_SURVEY_PROFILES',
                        'TOTAL_LENGTH_OF_SURVEY_PROFILES',
                        'INTERPOLATION_METHOD',
                        'INVESTIGATOR', 
                        'REFERENCES',
                        'MEAN_THICKNESS_UNCERTAINTY'
                       ], axis=1)
idx = glathida.index[glathida['GLACIER_DB'] == 'GLIMS']
# glathida = glathida.loc[idx]
# glathida['GLACIER_ID'] = glathida['GLACIER_ID'].str[6:]
# glathida["GLACIER_DB"].unique()
glathida.loc[idx]

,GlaThiDa_ID,GLACIER_DB,GLACIER_ID,LAT,LON,AREA,MEAN_SLOPE,MEAN_THICKNESS
604,2095,GLIMS,G007707E46420N,46.415400,7.708700,NaN,NaN,NaN
610,2101,GLIMS,G290838E77522N,77.522300,-69.161600,NaN,NaN,114.0
618,2109,GLIMS,G014140E77907N,77.907200,14.124400,2.84,NaN,81.0
619,2110,GLIMS,G014601E77917N,77.915200,14.601000,2.61,NaN,34.0
620,2111,GLIMS,G016068E78246N,78.245480,16.064380,2.10,13.0,32.0
621,2112,GLIMS,G014123E77878N,77.875600,14.100400,8.14,NaN,91.0
622,2113,GLIMS,G014442E77835N,77.835800,14.407700,47.30,NaN,106.0
623,2114,GLIMS,G014692E77911N,77.911100,14.687100,2.26,NaN,26.0
624,2115,GLIMS,G014983E77927N,77.926600,14.968600,6.24,NaN,29.0
625,2116,GLIMS,G015001E78014N,77.998900,15.030500,5.46,NaN,33.0


## B. Load and Organize the RGI datasets

In [3]:
# filename = '/home/sa42/data/glac/glims/glims_extra/dump2/07_rgi60_Svalbard.csv'
# with open(filename, 'rb') as file:
#     print(chardet.detect(file.read()))
df = pd.read_csv('/home/sa42/data/glac/glims/glims_extra/dump2/07_rgi60_Svalbard.csv',
                 encoding = 'ISO-8859-1')
RGI_07 = df
RGI_07 = RGI_07.drop([
    'GLIMSId',
    'BgnDate',
    'EndDate',
    'O1Region',
    'O2Region',
    'Status',
    'Connect',
    'Form',
    'TermType',
    'Surging',
    'Linkages',
    'Name',
],axis=1)

df = pd.read_csv('/home/sa42/data/glac/glims/glims_extra/dump2/10_rgi60_NorthAsia.csv',
                 encoding='ISO-8859-1')
RGI_10 = df
RGI_10 = RGI_10.drop([
    'GLIMSId',
    'BgnDate',
    'EndDate',
    'O1Region',
    'O2Region',
    'Status',
    'Connect',
    'Form',
    'TermType',
    'Surging',
    'Linkages',
    'Name',
],axis=1)

with open('/home/sa42/data/glac/glims/glims_extra/dump2/13_rgi60_CentralAsia.csv','r') as temp_f:
# get No of columns in each line
    col_count = [ len(l.split(',')) for l in temp_f.readlines() ]
### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]
### Read csv
df = pd.read_csv('/home/sa42/data/glac/glims/glims_extra/dump2/13_rgi60_CentralAsia.csv',
                 header=None, delimiter=',',names=column_names)
col_names = list(df.iloc[0])
df = pd.read_csv('/home/sa42/data/glac/glims/glims_extra/dump2/13_rgi60_CentralAsia.csv',
                 header=None, delimiter=',', names=col_names)
df.drop(0, inplace=True); df.reset_index(drop=True, inplace=True)
RGI_13 = df
RGI_13 = RGI_13.drop([
    'GLIMSId',
    'BgnDate',
    'EndDate',
    'O1Region',
    'O2Region',
    'Status',
    'Connect',
    'Form',
    'TermType',
    'Surging',
    'Linkages',
    'Name',
],axis=1)

df = pd.read_csv('/home/sa42/data/glac/glims/glims_extra/dump2/16_rgi60_LowLatitudes.csv',
                 encoding='ISO-8859-1')
RGI_16 = df
RGI_16 = RGI_16.drop([
    'GLIMSId',
    'BgnDate',
    'EndDate',
    'O1Region',
    'O2Region',
    'Status',
    'Connect',
    'Form',
    'TermType',
    'Surging',
    'Linkages',
    'Name',
],axis=1)

RGI = pd.concat([RGI_07,RGI_10,RGI_13,RGI_16])
RGI['RGIId'] = RGI['RGIId'].str[6:]

/home/sa42/miniconda3/envs/python-cartopy-f/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# C. Merge RGI and GlaThiDa along the RGI-ID
The output will include the above GlaThiDa fields, as well as Zmin, Zmax, Zmed, Aspect, and Lmax from RGI

In [4]:
idx = glathida.index[glathida['GLACIER_DB']=='RGI']
glathida = glathida.loc[idx]
glathida.rename(columns = {'GLACIER_ID':'RGIId'},inplace=True)

RGI_glathida = pd.merge(glathida,RGI, how = 'inner', on='RGIId')
RGI_glathida = RGI_glathida[[
    'LAT',
    'LON',
    'Aspect',
    'AREA',
    'MEAN_SLOPE',
    'MEAN_THICKNESS',
    'CenLon',
    'CenLat',
    'Area',
    'Zmin',
    'Zmax',
    'Zmed',
    'Slope',
    'Lmax',
    'MAXIMUM_THICKNESS',
]]
RGI_glathida = RGI_glathida.dropna()
RGI_glathida=RGI_glathida.apply(pd.to_numeric)

KeyError: "['MAXIMUM_THICKNESS'] not in index"

### separate out features - what will be trained to predict desired attribute

In [ ]:
#split the dataset and reserve some to test what was trained.
train_dataset = RGI_glathida.sample(frac=0.8, random_state=0)
test_dataset = RGI_glathida.drop(train_dataset.index)
train_features = train_dataset.copy()
test_features = test_dataset.copy()

#define label - attribute training to be picked
train_labels = train_features.pop('MAXIMUM_THICKNESS')
test_labels = test_features.pop('MAXIMUM_THICKNESS')
# train_features.describe().transpose()[['mean', 'std']]

# Step 2: Setup the Model

### Establish normalization layers

In [ ]:
normalizer = {}
variable_list = (
 'LAT',
 'LON',
 'AREA',
 'MEAN_SLOPE',
 'MEAN_THICKNESS',
 'CenLon',
 'CenLat',
 'Aspect',
 'Area',
 'Zmin',
 'Zmax',
 'Zmed',
 'Slope',
 'Lmax'
)
for variable_name in variable_list:

    normalizer[variable_name] = preprocessing.Normalization(input_shape=[1,], axis=None)
    normalizer[variable_name].adapt(np.array(train_features[variable_name]))
    
    
normalizer['ALL'] = preprocessing.Normalization(axis=-1)
normalizer['ALL'].adapt(np.array(train_features))

### Single variable linear regression functions

In [ ]:
# glacierml

def build_linear_model(normalizer):
    model = tf.keras.Sequential([
        normalizer,
        layers.Dense(1)
    ])

    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.1),
        loss='mean_absolute_error')
    
    return model

def plot_single_model_variable(x, y,feature_name):
    plt.scatter(train_features[feature_name], train_labels, label='Data')
    plt.plot(x, y, color='k', label='Predictions')
    plt.xlabel(feature_name)
    plt.ylabel('Avg Thickness (m)')
#     plt.xlim((0,20))
    plt.legend()
      
def plot_loss(history):
#     plt.subplots(figsize=(10,5))
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    #   plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

In [ ]:
test_results = {}
linear_model = {}
linear_history = {}
linear_results = {}

# Linear regression

In [ ]:
%%time
for variable_name in variable_list:

    linear_model[variable_name] = build_linear_model(normalizer[variable_name])
    linear_history[variable_name] = linear_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1000,
                                        verbose=0,
                                        validation_split = 0.2)
    
    
    linear_results[variable_name] = linear_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

linear_results

In [ ]:
for variable_name in variable_list:    
    df = pd.DataFrame(linear_history[variable_name].history)
    dfs = df.loc[[df.last_valid_index()]]
    dfs.insert(0, 'Variable', [variable_name])
    
    print(dfs)
    print('')


In [ ]:
fig,ax=plt.subplots(2,6,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    ax = plt.subplot(4,4,i+1)
    plt.subplot(4,4,i+1)
    plot_loss(linear_history[variable_name])
    ax.set_title(variable_name)
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_loss.eps")

# Linear regression with multiple inputs

In [ ]:
linear_model = build_linear_model(normalizer['ALL'])

history_full = linear_model.fit(
train_features, train_labels,        
   epochs=1000,
   verbose=0,
   validation_split = 0.2)

test_results['MULTI'] = linear_model.evaluate(
    test_features,
    test_labels, verbose=0)

test_results

In [ ]:
df = pd.DataFrame(history_full.history)
dfs = df.loc[[df.last_valid_index()]]
dfs.insert(0, 'Variable', 'Multi-Variable')
    
print(dfs)



In [ ]:
plot_loss(history_full)
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_full_loss.eps")

# DNN regression

## Define regression functions

In [ ]:
def build_dnn_model(norm):
    model = keras.Sequential([
              norm,
              layers.Dense(64, activation='relu'),
              layers.Dense(64, activation='relu'),
              layers.Dense(1) ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01))
    
    return model

def dnn_test_results(feature_name):
    dnn_test_results[feature_name] = dnn_model.evaluate(
        test_features[feature_name],
        test_labels, verbose=0)

In [ ]:
test_results = {}
dnn_model = {}
dnn_history = {}
dnn_results = {}

In [ ]:
for variable_name in variable_list:

    dnn_model[variable_name] = build_dnn_model(normalizer[variable_name])
    dnn_history[variable_name] = dnn_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1000,
                                        verbose=0,
                                        validation_split = 0.2)
    dnn_results[variable_name] = dnn_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

dnn_results

In [ ]:
for variable_name in variable_list:    
    df = pd.DataFrame(dnn_history[variable_name].history)
    dfs = df.loc[[df.last_valid_index()]]
    dfs.insert(0, 'Variable', [variable_name])
    
    print(dfs)
    print('')


In [ ]:
fig,ax=plt.subplots(4,4,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    plt.subplot(4,4,i+1)
    xmax = np.max(train_features[variable_name])
    xmin = np.min(train_features[variable_name])
    x = tf.linspace(xmin, xmax, 101)
    y = dnn_model[variable_name].predict(x)
    plot_single_model_variable(x,y,variable_name)

In [ ]:
fig,ax=plt.subplots(4,4,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    ax = plt.subplot(4,4,i+1)
    plot_loss(dnn_history[variable_name])
    ax.set_title(variable_name)
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_loss.eps")

# Full model

In [ ]:
dnn_full_model = build_dnn_model(normalizer['ALL'])

In [ ]:
%%time
dnn_history_full = dnn_full_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=1000)

In [ ]:
df = pd.DataFrame(dnn_history_full.history)
dfs = df.loc[[df.last_valid_index()]]
dfs.insert(0, 'Variable', 'Multi-Variable')
    
print(dfs)


In [ ]:
plot_loss(dnn_history_full)
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_full_loss.png")

In [ ]:
y = dnn_full_model.predict(test_features)
plt.plot(test_labels,y,'.')
plt.plot((0,200),(0,200),'-')
plt.xlabel('True Thickness (m)')
plt.ylabel('Model Thickness (m)')
plt.xlim((0,200))
plt.ylim((0,200))
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_res.EPS")